## TFIDF and ANN

In [ ]:
from keras import backend as K
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, Conv2D, MaxPooling1D, Dropout, Activation,GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.optimizers import Adam
from time import time
from keras.callbacks import TensorBoard
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam

In [ ]:
import pandas as pd
import csv
from scipy.sparse import coo_matrix, hstack
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
%matplotlib inline
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import ToktokTokenizer
lemma=WordNetLemmatizer()
token=ToktokTokenizer()
stop_words = set(stopwords.words('english'))

df = pd.read_csv('train.csv')
df['merged'] = df['TITLE']+" " + df['ABSTRACT']
df.drop(['ID','TITLE','ABSTRACT'], inplace = True, axis = 1)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,merged
0,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps ...
1,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation ...
2,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...
3,0,0,1,0,0,0,A finite element approximation for the stochas...
4,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...


In [ ]:
# function for text cleaning 
def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    return text

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w,'v')
        listLemma.append(x)
        listLemma.append(" ")
    return ''.join(listLemma)

In [ ]:
def PreProcessing(text):
    text=clean_text(text)
    text=remove_stopwords(text)
    # text=lemitizeWords(text)
    return(text)

In [ ]:
df['merged'] = df['merged'].apply(lambda com : PreProcessing(com))
df.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,merged
0,1,0,0,0,0,0,reconstructing subject specific effect maps pr...
1,1,0,0,0,0,0,rotation invariance neural network rotation in...
2,0,0,1,0,0,0,spherical polyharmonics poisson kernels polyha...
3,0,0,1,0,0,0,finite element approximation stochastic maxwel...
4,1,0,0,1,0,0,comparative study discrete wavelet transforms ...


In [ ]:
y = df.iloc[:,0:6]
y.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,1,0,0,1,0,0


In [ ]:
df_new = df.iloc[:,6:]
df_new.head()

,merged
0,reconstructing subject specific effect maps pr...
1,rotation invariance neural network rotation in...
2,spherical polyharmonics poisson kernels polyha...
3,finite element approximation stochastic maxwel...
4,comparative study discrete wavelet transforms ...


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(df_new['merged'], y, test_size=0.1, random_state=10)

In [ ]:
# tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=300000, smooth_idf=True,use_idf=True,ngram_range=(1,3),dtype=np.float32)

In [ ]:
# # create TF-IDF features
x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
x_val_tfidf = tfidf_vectorizer.transform(x_val)

In [ ]:
# #Using CountVectorizer
# countvector = CountVectorizer(ngram_range=(1,2))
# X_train_cv = countvector.fit_transform(x_train)
# X_test_cv = countvector.transform(x_val)

In [ ]:
# #Using TfidfVectorizer
# tfidfvector = TfidfTransformer()
# X_train_tf = tfidfvector.fit_transform(X_train_cv)
# X_test_tf = tfidfvector.fit_transform(X_test_cv)

In [ ]:
from keras.layers import  Dropout, Dense
from keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics

def TFIDF(X_train, X_test,MAX_NB_WORDS=300000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS,max_df=0.9, smooth_idf=True,use_idf=True,ngram_range=(1,3),dtype=np.float32)
    X_train = vectorizer_x.fit_transform(X_train)
    X_test = vectorizer_x.transform(X_test)
    return (X_train,X_test)

def Build_Model_DNN_Text(shape, nClasses, dropout=0.1):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 1024 # number of nodes
    nLayers = 4 # number of  hidden layer

    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(512,activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(216,activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(dropout))

    model.add(Dense(32,activation='relu'))
    model.add(Dropout(dropout))

    model.add(Dense(nClasses, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [ ]:
import tensorflow as tf
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.sparse.reorder(tf.SparseTensor(indices, coo.data, coo.shape))

y_train = tf.convert_to_tensor(y_train, bool)
y_val = tf.convert_to_tensor(y_val, bool)


In [ ]:
X_train_tfidf,X_test_tfidf = TFIDF(x_train,x_val)

model_DNN = Build_Model_DNN_Text(X_train_tfidf.shape[1], 6)
model_DNN.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              307201024 
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 216)               110808    
_________________________________________________________________
dropout_2 (Dropout)          (None, 216)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               2

In [ ]:
model_DNN.fit(convert_sparse_matrix_to_sparse_tensor(X_train_tfidf), y_train,
                              validation_data=(convert_sparse_matrix_to_sparse_tensor(X_test_tfidf), y_val),
                              epochs=50,
                              batch_size=128, verbose = 2)

Epoch 1/50
148/148 - 141s - loss: 0.3298 - accuracy: 0.5886 - val_loss: 0.1896 - val_accuracy: 0.7459
Epoch 2/50
148/148 - 140s - loss: 0.1386 - accuracy: 0.7927 - val_loss: 0.1885 - val_accuracy: 0.7660
Epoch 3/50
148/148 - 139s - loss: 0.0583 - accuracy: 0.8416 - val_loss: 0.2470 - val_accuracy: 0.7631
Epoch 4/50
148/148 - 139s - loss: 0.0303 - accuracy: 0.8668 - val_loss: 0.3136 - val_accuracy: 0.7769
Epoch 5/50
148/148 - 140s - loss: 0.0154 - accuracy: 0.8710 - val_loss: 0.3609 - val_accuracy: 0.7836
Epoch 6/50
148/148 - 139s - loss: 0.0100 - accuracy: 0.8595 - val_loss: 0.3879 - val_accuracy: 0.7626
Epoch 7/50
148/148 - 139s - loss: 0.0080 - accuracy: 0.8662 - val_loss: 0.4365 - val_accuracy: 0.7660
Epoch 8/50
148/148 - 139s - loss: 0.0056 - accuracy: 0.8635 - val_loss: 0.4589 - val_accuracy: 0.7812
Epoch 9/50
148/148 - 140s - loss: 0.0052 - accuracy: 0.8585 - val_loss: 0.4467 - val_accuracy: 0.7531
Epoch 10/50
148/148 - 139s - loss: 0.0045 - accuracy: 0.8681 - val_loss: 0.4770 - 

In [ ]:
# make predictions for validation set
y_pred = model_DNN.predict(convert_sparse_matrix_to_sparse_tensor(X_test_tfidf))
# evaluate performance

print('F1 score',f1_score(np.where(y_pred>0.5,1,0), y_val, average="micro"))
print('Accuracy',accuracy_score(np.where(y_pred>0.5,1,0), y_val))

F1 score 0.8255351392309148
Accuracy 0.6825548141086749


In [ ]:
def call_test_data():
    test_df = pd.read_csv('test.csv')
    test_df['merged'] = test_df['TITLE'] + ' ' + test_df['ABSTRACT']
    test_df['merged'] = test_df['merged'].apply(lambda x: PreProcessing(x))
    
    test_df_new = test_df['merged']
    return test_df_new

In [ ]:
data_test = call_test_data()
data_test.head()

0    closed form marginal likelihood gamma poisson ...
1    laboratory mid ir spectra equilibrated igneous...
2    case static amsdu aggregation wlans frame aggr...
3    gaia eso survey inner disk intermediate age op...
4    witness functions versus interpretation functi...
Name: merged, dtype: object

In [ ]:
X_train_tfidf,data_test_tfidf = TFIDF(x_train,data_test)

In [ ]:
# make predictions for validation set
y_test_pred = model_DNN.predict(convert_sparse_matrix_to_sparse_tensor(data_test_tfidf))
# evaluate performance
y_test_pred
# print('F1 score',f1_score(np.where(y_pred>0.5,1,0), y_val, average="micro"))
# print('Accuracy',accuracy_score(np.where(y_pred>0.5,1,0), y_val))

array([[9.8568201e-04, 4.7665688e-07, 8.6992979e-04, 9.9999714e-01,
        8.8609740e-06, 3.0741769e-06],
       [3.1563552e-15, 1.0000000e+00, 1.7303320e-11, 9.1336140e-12,
        4.5173718e-16, 1.8739562e-15],
       [9.4533801e-01, 1.0934740e-02, 1.7881483e-02, 9.3039739e-01,
        2.1320879e-02, 7.7667832e-04],
       ...,
       [8.4806424e-01, 1.7060608e-02, 9.7885132e-03, 4.8203889e-01,
        9.3704498e-01, 5.2706152e-02],
       [4.1578114e-03, 2.0454491e-09, 3.6415458e-04, 1.0000000e+00,
        7.5976828e-08, 1.9849957e-09],
       [9.9870443e-01, 9.2247128e-04, 1.0570288e-03, 4.0677190e-03,
        5.4152541e-05, 4.2409642e-06]], dtype=float32)

In [ ]:
y_test_pred = np.where(y_test_pred>0.5, 1,0)
y_test_pred

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0],
       ...,
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [ ]:
ans = pd.DataFrame(y_test_pred, columns = ['Computer Science', 'Physics', 'Mathematics', 'Statistics',
       'Quantitative Biology', 'Quantitative Finance'])
ans.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,0,0,0,1,0,0
1,0,1,0,0,0,0
2,1,0,0,1,0,0
3,0,1,0,0,0,0
4,1,0,0,0,0,0


In [ ]:
ans.to_csv('submissionv26.csv', index=False)

In [ ]:
# Got 0.834 f1_score on testing data